Trying to figure out how I can put the pickle files on git, it crashes 30 files in. will update ASAP

In [2]:
import torch

In [1]:
from models.yolo import Model, nn, np, amp
from utils.autoanchor import check_anchors
from utils.datasets import create_dataloader, letterbox
from utils.general import colorstr, increment_path, init_seeds, one_cycle, check_img_size, labels_to_class_weights, \
    strip_optimizer, xywh2xyxy, scale_coords
from utils.loss import ComputeLoss
from utils.metrics import fitness
from utils.torch_utils import select_device, logger, intersect_dicts, ModelEMA, de_parallel
import yaml
import pickle

In [3]:
pwd

'C:\\Experiments\\ALFL'

In [3]:
cl = torch.load("runs/run-cl-trial-0-0/weights/last.pt")

In [4]:
fl = torch.load("runs/run-fl-trial-3-0/weights/last.pt")

In [4]:
device = torch.device('cuda')

In [5]:
device

device(type='cuda')

In [18]:
objects = []
with (open("runs/run-flcl-cl-1epoch-feb23-0-0/weights/best.pickle", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [20]:
for param_tensor in w.state_dict():
    print(param_tensor, "\t", w.state_dict()[param_tensor].size())

AttributeError: 'dict' object has no attribute 'state_dict'

In [6]:
state_dict_cl_feb24_last = torch.load("runs/run-flcl-cl-1epoch-feb24-0-0/weights/last.pt", map_location=device)

In [7]:
weights_cl_feb24_last = state_dict_cl_feb24_last['model'].float().state_dict()

In [8]:
state_dict_fl_feb24_last = torch.load("runs/run-flcl-fl-1epoch-last-feb24-3-0/weights/best.pt", map_location=device)

In [9]:
weights_fl_feb24_last = state_dict_fl_feb24_last['model'].float().state_dict()

In [13]:
cl.keys()

dict_keys(['epoch', 'best_fitness', 'training_results', 'model', 'ema', 'updates', 'optimizer', 'wandb_id'])

In [41]:
cl_weights = cl['model'].float().state_dict()
fl_weights = fl['model'].float().state_dict()

In [6]:
cl_weights['model.0.conv.bn.bias']

tensor([ 0.03793, -0.00051,  0.01470, -0.02090, -0.01141, -0.00566, -0.01051, -0.01109, -0.00793, -0.01720,  0.01021, -0.01122, -0.02608, -0.00031, -0.00557,  0.01715, -0.00371, -0.00633,  0.01295,  0.00765,  0.00310,  0.00081, -0.00083,  0.01457, -0.01041,  0.00020,  0.00790, -0.00282, -0.00309,  0.01105,  0.03296,
        -0.01025])

In [7]:
fl_weights['model.0.conv.bn.bias']

tensor([-1.45569e-02,  4.49753e-03, -8.75592e-05, -1.34659e-02, -3.80898e-03,  9.83429e-03,  8.54015e-04, -3.68595e-04,  1.50323e-04, -8.74996e-05,  2.85339e-03, -5.47791e-03, -8.00323e-03, -1.27640e-02,  5.93662e-04, -1.01395e-02, -3.96729e-03,  1.23062e-02, -8.33511e-04,  1.15509e-02, -7.57980e-03, -2.78282e-03,
        -1.37329e-03, -4.76456e-03, -6.40392e-04,  4.11606e-03,  9.26971e-04,  6.60324e-03, -5.90134e-03, -1.63078e-03,  9.69696e-03,  1.74999e-03])

In [21]:
sum((x - y).abs().sum() for x, y in zip(cl_weights['model.0.conv.bn.bias'], fl_weights['model.0.conv.bn.bias']))

tensor(0.37875)

In [24]:
-0.01025-1.74999e-03

-0.01199999

In [22]:
diff_clfl = cl_weights['model.0.conv.bn.bias']-fl_weights['model.0.conv.bn.bias']
print(diff_clfl)

tensor([ 0.05249, -0.00501,  0.01479, -0.00744, -0.00760, -0.01550, -0.01136, -0.01072, -0.00808, -0.01711,  0.00735, -0.00574, -0.01807,  0.01245, -0.00616,  0.02729,  0.00025, -0.01864,  0.01379, -0.00390,  0.01068,  0.00359,  0.00055,  0.01934, -0.00977, -0.00391,  0.00697, -0.00942,  0.00281,  0.01269,  0.02326,
        -0.01200])


In [46]:
td

tensor([ 0.05249, -0.00501,  0.01479, -0.00744, -0.00760, -0.01550, -0.01136, -0.01072, -0.00808, -0.01711,  0.00735, -0.00574, -0.01807,  0.01245, -0.00616,  0.02729,  0.00025, -0.01864,  0.01379, -0.00390,  0.01068,  0.00359,  0.00055,  0.01934, -0.00977, -0.00391,  0.00697, -0.00942,  0.00281,  0.01269,  0.02326,
        -0.01200])

In [30]:
list(enumerate(cl_weights['model.13.cv3.conv.weight']))

[(0,
  tensor([[[-0.03677]],
  
          [[ 0.01854]],
  
          [[ 0.03729]],
  
          [[-0.02916]],
  
          [[-0.02515]],
  
          [[ 0.03174]],
  
          [[ 0.04349]],
  
          [[-0.04266]],
  
          [[-0.04520]],
  
          [[-0.02216]],
  
          [[ 0.05786]],
  
          [[-0.03522]],
  
          [[-0.05258]],
  
          [[-0.00958]],
  
          [[ 0.02567]],
  
          [[ 0.01282]],
  
          [[-0.03793]],
  
          [[ 0.06226]],
  
          [[ 0.05255]],
  
          [[-0.03290]],
  
          [[ 0.05096]],
  
          [[ 0.03476]],
  
          [[ 0.05698]],
  
          [[-0.05936]],
  
          [[ 0.04266]],
  
          [[-0.00565]],
  
          [[-0.01631]],
  
          [[-0.01569]],
  
          [[-0.00410]],
  
          [[ 0.02916]],
  
          [[ 0.04861]],
  
          [[-0.02272]],
  
          [[-0.00613]],
  
          [[ 0.01131]],
  
          [[ 0.04092]],
  
          [[-0.03510]],
  
          [[-0.05228]],

In [27]:
diff={}
diff_0={}
for k in cl_weights:
    diff[k] = cl_weights[k]-fl_weights[k]

RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [52]:
diff

{'model.0.conv.conv.weight': tensor([[[[-3.05176e-05,  0.00000e+00,  0.00000e+00],
           [-3.05176e-05,  0.00000e+00,  0.00000e+00],
           [-1.52588e-05, -3.05176e-05, -2.28882e-05]],
 
          [[ 0.00000e+00,  1.52588e-05,  1.90735e-05],
           [-3.05176e-05,  0.00000e+00,  3.81470e-06],
           [-7.62939e-06, -7.62939e-06, -1.07288e-05]],
 
          [[ 0.00000e+00,  1.52588e-05,  3.05176e-05],
           [-3.05176e-05,  0.00000e+00,  3.05176e-05],
           [-1.52588e-05, -1.14441e-05,  0.00000e+00]],
 
          ...,
 
          [[ 0.00000e+00, -3.81470e-06, -1.52588e-05],
           [-3.05176e-05,  0.00000e+00,  0.00000e+00],
           [ 0.00000e+00, -3.05176e-05, -3.05176e-05]],
 
          [[-1.52588e-05,  0.00000e+00,  0.00000e+00],
           [-1.52588e-05,  0.00000e+00,  0.00000e+00],
           [ 0.00000e+00, -1.52588e-05,  0.00000e+00]],
 
          [[-1.52588e-05,  0.00000e+00,  1.52588e-05],
           [-3.05176e-05,  0.00000e+00,  0.00000e+00],
     

In [33]:
device = torch.device('cuda')

In [31]:
with open('hyps/hyp.scratch.yaml') as f:
    hyp = yaml.safe_load(f)

In [34]:
model = Model('models/yolov5s.yaml', ch=3, nc=8, anchors=hyp.get('anchors')).to(device)

In [35]:
model.info(verbose=True)

layer                                     name  gradient   parameters                shape         mu      sigma
    0                 model.0.conv.conv.weight      True         3456       [32, 12, 3, 3]   0.000636      0.055
    1                   model.0.conv.bn.weight      True           32                 [32]          1          0
    2                     model.0.conv.bn.bias      True           32                 [32]          0          0
    3                      model.1.conv.weight      True        18432       [64, 32, 3, 3]  -0.000174     0.0342
    4                        model.1.bn.weight      True           64                 [64]          1          0
    5                          model.1.bn.bias      True           64                 [64]          0          0
    6                  model.2.cv1.conv.weight      True         2048       [32, 64, 1, 1]    0.00118     0.0721
    7                    model.2.cv1.bn.weight      True           32                 [32]      

In [42]:
def compare_models(model_1, model_2):
    models_differ = 0
    for key_item_1, key_item_2 in zip(model_1.items(), model_2.items()):
        if torch.equal(key_item_1[1], key_item_2[1]):
            pass
        else:
            models_differ += 1
            if (key_item_1[0] == key_item_2[0]):
                print('Mismtach found at', key_item_1[0])
            else:
                raise Exception
    if models_differ == 0:
        print('Models match perfectly! :)')

In [46]:
compare_models(cl_weights, fl_weights)

Mismtach found at model.0.conv.conv.weight
Mismtach found at model.0.conv.bn.bias
Mismtach found at model.0.conv.bn.running_mean
Mismtach found at model.0.conv.bn.running_var
Mismtach found at model.1.conv.weight
Mismtach found at model.1.bn.bias
Mismtach found at model.1.bn.running_mean
Mismtach found at model.1.bn.running_var
Mismtach found at model.2.cv1.conv.weight
Mismtach found at model.2.cv1.bn.bias
Mismtach found at model.2.cv1.bn.running_mean
Mismtach found at model.2.cv1.bn.running_var
Mismtach found at model.2.cv2.conv.weight
Mismtach found at model.2.cv2.bn.bias
Mismtach found at model.2.cv2.bn.running_mean
Mismtach found at model.2.cv2.bn.running_var
Mismtach found at model.2.cv3.conv.weight
Mismtach found at model.2.cv3.bn.bias
Mismtach found at model.2.cv3.bn.running_mean
Mismtach found at model.2.cv3.bn.running_var
Mismtach found at model.2.m.0.cv1.conv.weight
Mismtach found at model.2.m.0.cv1.bn.bias
Mismtach found at model.2.m.0.cv1.bn.running_mean
Mismtach found at m